In [1]:
# Import system packages
using Printf
using DelimitedFiles
using Plots
using LaTeXStrings

In [ ]:
#            Psurf/kbar         Msurf/Me            Rsurf/Re            Fint/ Wm-2
structure = [36.606020371988    1.89635484051907    1.26977486802543    0.1
             72.7858858655158   1.87155246958974    1.19269414869405    0.1
             203.510277091369   1.82923648889318    1.03435826758376    0.1    ]
nsamps = size(structure)[1]

In [3]:
# Import AGNI
ROOT_DIR = abspath(joinpath(pwd(),"../"))
using AGNI
import AGNI.atmosphere as atmosphere
import AGNI.solver as nl
import AGNI.setpt as setpt
import AGNI.dump as dump
import AGNI.plotting as plotting

# Disable logging from AGNI module
AGNI.setup_logging("",1)

In [ ]:
R_earth = 6.371e6
M_earth = 5.972e24

In [ ]:
# Configuration options
instellation    = 6501.0
mass            = 1.873*M_earth
radius          = 1.265*R_earth
gravity         = 6.67e-11 * mass / radius^2
p_surf          = 5.57e4

s0_fact         = 0.25
zenith_degrees  = 54.74
albedo_b        = 0.0
nlev_centre     = 52
p_top           = 1e-5      # bar
mole_fractions  = Dict([
                        ("H2S", 0.23988329190194904),
                        ("H2O", 1e-2),
                        # ("SO2", 0.0044668359215096305),
                        # ("N2" , 5.623413251903491e-6),
                        ("H2" , 0.7093907341517219)
                    ])
tmp_surf        = 3100.0
thermo          = true

albedo_data   = joinpath(ROOT_DIR,"res/surface_albedos/lunar_marebasalt.dat")
spectral_file = joinpath(ROOT_DIR,"res/spectral_files/Honeyside/256/Honeyside.sf")
star_file     = joinpath(ROOT_DIR,"res/stellar_spectra/l-98-59.txt")
output_dir    = joinpath(ROOT_DIR,"out/")

In [6]:
# Create output directory
rm(output_dir,force=true,recursive=true)
mkdir(output_dir);

In [ ]:
# Setup atmosphere
atmos = atmosphere.Atmos_t()
atmosphere.setup!(atmos, ROOT_DIR, output_dir,
                        spectral_file,
                        instellation, s0_fact, albedo_b, zenith_degrees,
                        tmp_surf,
                        gravity, radius,
                        nlev_centre, p_surf, p_top,
                        mole_fractions, "",

                        flux_int = 0.0,
                        flag_gcontinuum=true,
                        flag_rayleigh=true,
                        thermo_functions=thermo,
                        surface_material=albedo_data
                    )
atmosphere.allocate!(atmos, star_file)

In [8]:
setpt.loglinear!(atmos, 700.0)

In [ ]:
solver_success = nl.solve_energy!(atmos,
                                    sol_type=1,
                                    sens_heat=true,
                                    latent=true,
                                    method=1,
                                    dx_max=400.0,
                                    ls_method=0,
                                    save_frames=false, modplot=1,
                                    conv_atol=0.5,
                                    modprint=1
                                    )
println("Solver success? $solver_success")
atm_base = deepcopy(atmos);

In [ ]:
atm_arr = atmosphere.Atmos_t[]

@printf("Running model for %d samples... \n", nsamps)

# atmos = deepcopy(atm_base)

setpt.isothermal!(atmos, 1000.0)
for i in 1:nsamps
    @printf("    sample %d/%d \n",i,nsamps)

    # update planet structure
    row = structure[i,:]
    Ps = row[1] * 1e3 * 1e5 # convert kbar -> bar -> Pa
    Ms = row[2] * M_earth
    Rs = row[3] * R_earth
    Fi = row[4]

    atmos.rp = Rs
    atmos.p_boa = Ps
    atmos.grav_surf = 6.67e-11 * Ms / Rs^2
    atmos.flux_int = Fi
    @printf("    Ps=%.2e Pa  ,  Ms=%.2e kg  ,  Rs=%.2e m  ,  Fi=%.3f Wm-2\n",Ps,Ms,Rs,Fi)

    atmosphere.generate_pgrid!(atmos)

    # Run model
    solver_success = nl.solve_energy!(atmos,
                                    sol_type=3,
                                    method=1,
                                    dx_max=300.0,
                                    ls_method=0,
                                    save_frames=false, modplot=1,
                                    modprint=1, perturb_all=true,
                                    conv_atol=1.0e-2
                                    )

    # Store result
    push!(atm_arr, deepcopy(atmos))
    @printf("--------------------------------- \n")
end
println("Done!")

In [ ]:
for (i, atm) in enumerate(atm_arr)
    @printf("Writing index %d \n",i)
    dump.write_ncdf(atm, joinpath(atm.OUT_DIR, "$i.nc"))
end
@printf("Done! \n")

In [ ]:
fs=12
lw=1.8
al=0.8

In [ ]:
# find highest psurf
psurf_max::Float64 = 1e3
psurf_idx::Int = 1
for i in 1:nsamps
    this_atm = atm_arr[i]
    if atm_arr[i].pl[end] > psurf_max
        psurf_max = atm_arr[i].pl[end]
        psurf_idx = i
    end
end

# y ticks
arr_P = atm_arr[psurf_idx].pl .* 1.0e-5 # Convert Pa to bar
ylims  = (arr_P[1]/2, arr_P[end]*2)
yticks = 10.0 .^ round.(Int,range( log10(ylims[1]), stop=log10(ylims[2]), step=1))

plt = plot(framestyle=:box, size=(750,500), dpi=300,
                leg=:outertopright, legcolumn=-1,
                tickfontsize=fs, guidefontsize=fs, legendfontsize=fs,
                legendtitlefontsize=fs,
                leg_title="Index",
                left_margin=4Plots.mm, bottom_margin=4Plots.mm)


# plot the profiles
for i in 1:nsamps
    this_atm = atm_arr[i]
    plot!(plt, this_atm.tmpl, this_atm.pl* 1.0e-5, linewidth=lw, label=@sprintf("%d",i), linealpha=al)
end

# decorate
xlabel!(plt, "Temperature[K]")
xaxis!(plt, minorgrid=true)
ylabel!(plt, "Pressure [bar]")
yflip!(plt)
yaxis!(plt, yscale=:log10, ylims=ylims, yticks=yticks)
display(plt)
savefig(plt,joinpath(output_dir,"profiles.pdf"))

In [ ]:
arr_P = atmos.pl .* 1.0e-5 # Convert Pa to bar
ylims  = (arr_P[1], arr_P[end])
yticks = 10.0 .^ round.(Int,range( log10(ylims[1]), stop=log10(ylims[2]), step=1))

plt = plot(framestyle=:box, size=(750,500), dpi=300,
                leg=:outertopright, legcolumn=-1,
                tickfontsize=fs, guidefontsize=fs, legendfontsize=fs,
                legendtitlefontsize=fs,
                leg_title="Index",
                left_margin=4Plots.mm, bottom_margin=4Plots.mm)


for i in 1:nsamps
    this_atm = atm_arr[i]
    plot!(plt, this_atm.flux_cdry[1:end-2], this_atm.pl[1:end-2]* 1.0e-5,  linewidth=lw, label=@sprintf("%d",i), linealpha=al)
end

xlabel!(plt, "Convective flux [W m-2]")
xaxis!(plt, minorgrid=true)
ylabel!(plt, "Pressure [bar]")
yflip!(plt)
yaxis!(plt, yscale=:log10, ylims=ylims, yticks=yticks)
display(plt)
savefig(plt,joinpath(output_dir,"convection.pdf"))

In [ ]:
arr_P = atmos.pl .* 1.0e-5 # Convert Pa to bar
ylims  = (arr_P[1], arr_P[end])
yticks = 10.0 .^ round.(Int,range( log10(ylims[1]), stop=log10(ylims[2]), step=1))

plt = plot(framestyle=:box, size=(750,500), dpi=300,
                leg=:outertopright, legcolumn=-1,
                tickfontsize=fs, guidefontsize=fs, legendfontsize=fs,
                legendtitlefontsize=fs,
                leg_title="Index",
                left_margin=4Plots.mm, bottom_margin=4Plots.mm)


for i in 1:nsamps
    this_atm = atm_arr[i]
    plot!(plt, this_atm.flux_d_sw, this_atm.pl*1.0e-5,  linewidth=lw, label=@sprintf("%d",i), linealpha=al)
end

xlabel!(plt, "Stellar flux [W m-2]")
xaxis!(plt, minorgrid=true)
ylabel!(plt, "Pressure [bar]")
yflip!(plt)
yaxis!(plt, yscale=:log10, ylims=ylims, yticks=yticks)
display(plt)
savefig(plt,joinpath(output_dir,"fl_D_SW.pdf"))

In [ ]:
plt = plot(framestyle=:box, size=(900,400), dpi=300,
                leg=:outertopright, legcolumn=-1,
                tickfontsize=fs, guidefontsize=fs, legendfontsize=fs,
                legendtitlefontsize=fs,
                leg_title="Index",
                left_margin=4Plots.mm, bottom_margin=4Plots.mm)

for i in 1:nsamps
    this_atm = atm_arr[i]


    # band widths
    wd = atmos.bands_wid * 1e9 # convert to nm

    # band centres
    xe = atmos.bands_cen * 1e9 # convert to nm

    # TOA upward spectral flux [erg s-1 cm-2 nm-1]
    yt = (this_atm.band_u_sw[1, :] .+  this_atm.band_u_lw[1, :]) .* 1000.0 ./ wd

    plot!(plt, xe, yt,  linewidth=lw, label=@sprintf("%d",i), linealpha=al)
end

xlabel!(plt, "Wavelength [nm]")
xaxis!(plt, xscale=:lo010, minorgrid=true, xlims=(300.0, 6.0e3))

ylabel!(plt, "Flux density [W m-2 nm-1]")
yaxis!(plt, yscale=:log10, ylims=(0.1, Inf))

display(plt)
savefig(plt,joinpath(output_dir,"emission.pdf"))